In [ ]:
!pip install pyheif

from PIL import Image
import pyheif

from tqdm.notebook import tqdm

from pathlib import Path
from tqdm import tqdm
import numpy as np

import PIL.Image as Image
import cv2
import torch
import torchvision
from IPython.display import display, clear_output
from sklearn.model_selection import train_test_split
from torch import nn
from torchvision import models, transforms as T
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import rc
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10

import os

clear_output()
from google.colab import drive
drive.mount('/content/drive')

model = torch.hub.load('ultralytics/yolov5', 'custom', path='ПУТЬ К ВЕСАМ')

In [ ]:
import random

def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_all(0)

In [ ]:
path = 'путь к фото'

class_0_title = 'класс 0'
class_0_folders = ['папки для 0 класса']

class_1_title = 'класс 1'
class_1_folders = ['папки для 1 класса']

class_2_title = 'класс 2'
class_2_folders = ['папки для 2 класса']

class_0_imgs = []
class_1_imgs = []
class_2_imgs = []

print('\tclass 0')
for i, folder in enumerate(class_0_folders):
    print('folder', i + 1, '/', len(class_0_folders))
    for file in tqdm(os.listdir(path=path + '/' + class_0_title + '/' + folder)):
        try:
            heif_file = pyheif.read(path + '/' + class_0_title + '/' + folder + '/' + file)
            image = Image.frombytes(
                heif_file.mode,
                heif_file.size,
                heif_file.data,
                "raw",
                heif_file.mode,
                heif_file.stride,
            ).resize((416, 416))
            class_0_imgs.append(image)
        except:
            image = Image.open(path + '/' + class_0_title + '/' + folder + '/' + file).resize((416, 416))
            class_0_imgs.append(image)

print('\tclass 1')
for i, folder in enumerate(class_1_folders):
    print('folder', i + 1, '/', len(class_1_folders))
    for file in tqdm(os.listdir(path=path + '/' + class_1_title + '/' + folder)):
        try:
            heif_file = pyheif.read(path + '/' + class_1_title + '/' + folder + '/' + file)
            image = Image.frombytes(
                heif_file.mode,
                heif_file.size,
                heif_file.data,
                "raw",
                heif_file.mode,
                heif_file.stride,
            ).resize((416, 416))
            class_1_imgs.append(image)
        except:
            image = Image.open(path + '/' + class_1_title + '/' + folder + '/' + file).resize((416, 416))
            class_1_imgs.append(image)

print('\tclass 2')
for i, folder in enumerate(class_2_folders):
    print('folder', i + 1, '/', len(class_2_folders))
    for file in tqdm(os.listdir(path=path + '/' + class_2_title + '/' + folder)):
        try:
            heif_file = pyheif.read(path + '/' + class_2_title + '/' + folder + '/' + file)
            image = Image.frombytes(
                heif_file.mode,
                heif_file.size,
                heif_file.data,
                "raw",
                heif_file.mode,
                heif_file.stride,
            ).resize((416, 416))
            class_2_imgs.append(image)
        except:
            image = Image.open(path + '/' + class_2_title + '/' + folder + '/' + file).resize((416, 416))
            class_2_imgs.append(image)

In [ ]:
drive.flush_and_unmount()

In [ ]:
os.mkdir('class0')
os.mkdir('class1')
os.mkdir('class2')

In [ ]:
for i, image in enumerate(class_0_imgs):
    image.save('class0/' + str(i) + '.jpg')
for i, image in enumerate(class_1_imgs):
    image.save('class1/' + str(i) + '.jpg')
for i, image in enumerate(class_2_imgs):
    image.save('class2/' + str(i) + '.jpg')

In [ ]:
preds_all = [[], [], []]
dirs = ['class0', 'class1', 'class2']
imgs_lens = [len(class_0_imgs), len(class_1_imgs), len(class_2_imgs)]
for j, dir in enumerate(dirs):
    inds = np.arange(imgs_lens[j])
    inds1 = inds[ : len(inds) // 100 * 100].reshape((-1, 100))
    inds2 = inds[len(inds) // 100 * 100:]
    for k in range(inds1.shape[0]):
        imgs = [dir + '/' + str(num) + '.jpg' for num in inds1[k]]
        model(imgs).save(save_dir='save_dir')
        preds = model(imgs).pred
        for i in range(len(preds)):
            preds[i] = preds[i].cpu()
        preds_all[j] += preds
    imgs = [dir + '/' + str(num) + '.jpg' for num in inds2]
    # model(imgs).save(save_dir='save_dir')
    preds = model(imgs).pred
    for i in range(len(preds)):
        preds[i] = preds[i].cpu()
    preds_all[j] += preds

In [ ]:
X = []
for preds_per_class in preds_all:
    X += preds_per_class
y = np.array([0] * len(class_0_imgs) + [1] * len(class_1_imgs) + [2] * len(class_2_imgs))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, x, y, max_len):
        self.x = x
        lens = []
        for i in range(len(self.x)):
            if self.x[i].shape[0]:
                self.x[i] = self.x[i][:, 4:]
            else:
                self.x[i].reshape((0, 2))
            conf = self.x[i][:, 0]
            argsort = torch.argsort(conf, dim=0, descending=True)
            self.x[i] = self.x[i].flatten()[: max_len * 2].tolist()
            self.x[i] += [0.] * (max_len * 2 - len(self.x[i])) + [len(self.x[i])]
            
        self.y = y.reshape((y.shape[0], 1))
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)
    
    def __len__(self):
        return len(self.x)

In [ ]:
BATCH_SIZE = 200

max_len = 50

train_dataset = MyDataset(X_train, y_train, max_len)
test_dataset = MyDataset(X_test, y_test, max_len)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(
           nn.Linear(max_len * 2 + 1, 200),
           nn.ReLU(),
           nn.Linear(200, 150),
           nn.ReLU(),
           nn.Linear(150, 3),
        )
    def forward(self, x):
        return self.linear(x)

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model = torch.load('model_params.pt', map_location=device)
model = model.to(device)

criterion = nn.CrossEntropyLoss(weight=torch.tensor([y.shape[0] / (y==0).sum(), y.shape[0] / (y==1).sum(), y.shape[0] / (y==2).sum()], dtype=torch.float).to(device))

In [ ]:
from IPython.display import display
from time import time

num_matches = 0
model.eval()
for x, y in tqdm(test_dataloader):
    with torch.no_grad():
            
        x = x.to(device)
        y = y.type(torch.LongTensor).squeeze(1)
        logits = model(x).cpu()
        num_matches += (np.argmax(logits, 1) == y).sum()
        
        
print('test accuracy = ', num_matches / len(test_dataset))

num_matches = 0
model.eval()
for x, y in tqdm(train_dataloader):
    with torch.no_grad():
            
        x = x.to(device)
        y = y.type(torch.LongTensor).squeeze(1)
        logits = model(x).cpu()
        num_matches += (np.argmax(logits, 1) == y).sum()
print('')
print('train accuracy = ', num_matches / len(train_dataset))